In [3]:
import sys
from pathlib import Path

# Point this at your QEPC project folder
project_root = Path(r"C:\Users\wdors\qepc_project")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("PYTHONPATH now includes:", project_root)


PYTHONPATH now includes: C:\Users\wdors\qepc_project


In [4]:
import time
import requests
import pandas as pd
from pathlib import Path

BASE_URL = "https://site.api.espn.com/apis/site/v2/sports/basketball/nba/injuries"

# Basic list of team codes; we can refine later
TEAM_CODES = [
    "ATL", "BOS", "BKN", "CHA", "CHI", "CLE", "DAL", "DEN", "DET", "GS",
    "HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NO", "NY",
    "OKC", "ORL", "PHI", "PHX", "POR", "SAC", "SA", "TOR", "UTA", "WAS",
]

def fetch_team_injuries(team_code):
    """Fetch injury info for a single NBA team from ESPN."""
    resp = requests.get(BASE_URL, params={"team": team_code}, timeout=10)
    resp.raise_for_status()
    data = resp.json()

    rows = []

    # ESPN's structure: leagues -> teams -> injuries
    for league in data.get("leagues", []):
        for team_block in league.get("teams", []):
            team_info = team_block.get("team", {})
            team_name = team_info.get("displayName") or team_info.get("name") or team_code

            for inj in team_block.get("injuries", []):
                athlete = inj.get("athlete", {})
                status = inj.get("status", {})

                rows.append(
                    {
                        "TeamCode": team_code,
                        "Team": team_name,
                        "PlayerName": athlete.get("displayName"),
                        "Position": (athlete.get("position") or {}).get("abbreviation"),
                        "StatusShort": (status.get("type") or {}).get("name"),
                        "StatusDetail": status.get("detail"),
                        "ESPNStatus": (status.get("type") or {}).get("description"),
                    }
                )
    return rows

all_rows = []

for code in TEAM_CODES:
    print(f"Fetching injuries for {code}...")
    try:
        rows = fetch_team_injuries(code)
        all_rows.extend(rows)
    except Exception as e:
        print(f"  Error fetching {code}: {e}")
    time.sleep(0.3)  # polite delay

df = pd.DataFrame(all_rows)

# Save into your project's data folder
project_root = Path.cwd()  # assuming you're running this from qepc_project root
out_path = project_root / "data" / "Injury_Overrides_espn.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(out_path, index=False)
print(f"\nSaved {len(df)} injury rows to {out_path}")


Fetching injuries for ATL...
Fetching injuries for BOS...
Fetching injuries for BKN...
Fetching injuries for CHA...
Fetching injuries for CHI...
Fetching injuries for CLE...
Fetching injuries for DAL...
Fetching injuries for DEN...
Fetching injuries for DET...
Fetching injuries for GS...
Fetching injuries for HOU...
Fetching injuries for IND...
Fetching injuries for LAC...
Fetching injuries for LAL...
Fetching injuries for MEM...
Fetching injuries for MIA...
Fetching injuries for MIL...
Fetching injuries for MIN...
Fetching injuries for NO...
Fetching injuries for NY...
Fetching injuries for OKC...
Fetching injuries for ORL...
Fetching injuries for PHI...
Fetching injuries for PHX...
Fetching injuries for POR...
Fetching injuries for SAC...
Fetching injuries for SA...
Fetching injuries for TOR...
Fetching injuries for UTA...
  Error fetching UTA: 400 Client Error: Bad Request for url: https://site.api.espn.com/apis/site/v2/sports/basketball/nba/injuries?team=UTA
Fetching injuries for W

In [5]:
import pandas as pd
from pathlib import Path
from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

# 1) Build your base team strengths
advanced_strengths = calculate_advanced_strengths()
advanced_team_strengths = (
    advanced_strengths
    .groupby("Team", as_index=False)
    .mean(numeric_only=True)
)

# 2) Load ESPN injury CSV that you generated locally
project_root = Path.cwd()
inj_path = project_root / "data" / "Injury_Overrides_espn.csv"
injuries = pd.read_csv(inj_path)

injuries.head()


[QEPC Paths] Project Root set: C:\Users\wdors\qepc_project
[QEPC Strength V2] Starting Advanced Calculation (Cutoff: Now)...
[QEPC PlayerData] Successfully loaded 1635462 rows from PlayerStatistics.csv.
[QEPC Opponent Processor] Loading raw team data for Weighted DRtg...


KeyboardInterrupt: 